<a href="https://colab.research.google.com/github/teatime77/xbrl-reader/blob/master/notebook/TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Matplotlibで日本語が表示できるようにします。

#### IPAフォントをインストールします。

In [0]:
!apt-get -y install fonts-ipafont-gothic

#### Matplotlibのフォントのキャッシュを再構築します。

In [0]:
import matplotlib
matplotlib.font_manager._rebuild()

#### <font color="red">キャッシュの再構築を有効にするために、ここでランタイムを再起動してください。</font>

#### <font color="red">以下の中から予測したい項目のコメントをはずしてください。</font>

In [0]:
target = '売上高'
# target = '営業利益'
# target = '経常利益'
# target = '税引前純利益'

### 選択した項目に対応するファイルをダウンロードします。

In [0]:
if   target == '売上高':
    ! wget http://lkzf.info/xbrl/data/2020-04-08/preprocess-uriage.pickle

elif target == '営業利益':
    ! wget http://lkzf.info/xbrl/data/2020-04-08/preprocess-eigyo.pickle

elif target == '経常利益':
    ! wget http://lkzf.info/xbrl/data/2020-04-08/preprocess-keijo.pickle

elif target == '税引前純利益':
    ! wget http://lkzf.info/xbrl/data/2020-04-08/preprocess-jun.pickle

### 必要なライブラリをインポートします。

In [0]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import datetime
import seaborn as sns
import pandas as pd
import numpy as np
import pickle
import pytz

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

sns.set(font='IPAGothic')

## TensorBoardの準備をします。

#### TensorBoardのノートブック拡張をロードします。

In [0]:
%load_ext tensorboard

### ログの出力先のフォルダーをクリアします。

In [0]:
!rm -rf ./logs/ 
!ls

### TensorBoardを開始します。

In [0]:
%tensorboard --logdir logs

## 全結合ニューラルネットワーク

#### データを読み込みます。

In [0]:
def read_data():

    if   target == '売上高':
        file_name = 'preprocess-uriage.pickle'
    elif target == '営業利益':
        file_name = 'preprocess-eigyo.pickle'
    elif target == '経常利益':
        file_name = 'preprocess-keijo.pickle'
    elif target == '税引前純利益':
        file_name = 'preprocess-jun.pickle'
    else:
        assert False

    with open(file_name, 'rb') as f:
        data = pickle.load(f)

    # PandasのDataFrame
    df = data['data_frame']

    # 予測する列の名前
    y_column = data['y_column']

    return df, y_column

In [0]:
df, y_column = read_data()

### データの準備をします。

In [0]:
X_columns = [ x for x in df.columns if x != y_column ]

# トレーニングデータとテストデータに分けます。
X_train, X_test, y_train, y_test = train_test_split(df[X_columns], df[y_column], test_size=0.2, random_state=0)

# numpyの配列を取り出します。
X_train = X_train.values
X_test  = X_test.values
y_train = y_train.values
y_test  = y_test.values

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### 全結合のモデルを作ります。

In [0]:
from keras.optimizers import Adam

unit = X_train.shape[1]

model = keras.Sequential([
    layers.Dense(unit, activation='relu', input_shape=[unit]),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(unit, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(unit, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(loss='mse', optimizer=optimizer, metrics=['mse', 'mae'])

### ログとEarlyStoppingのコールバックの設定をします。

In [0]:
log_dir = "logs/全結合-" + datetime.datetime.now(pytz.timezone('Asia/Tokyo')).strftime("%H時%M分%S秒")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)

### 学習を開始します。

In [0]:
model.fit(x=X_train, y=y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[tensorboard_callback, early_stop])

### テストデータで予測します。

In [0]:
y_pred = model.predict(X_test).flatten()

#### 平均二乗誤差と平均絶対誤差を表示します。

In [0]:
# 平均二乗誤差を計算します。
mse_pred = mean_squared_error(y_test, y_pred)
mse_mean = mean_squared_error(y_test, [y_test.mean()] * len(y_test) )

# 平均絶対誤差を計算します。
mae_pred = mean_absolute_error(y_test, y_pred)
mae_mean = mean_absolute_error(y_test, [y_test.mean()] * len(y_test) )

print('\n平均二乗誤差 : %.4f ( %.4f )  平均絶対誤差 : %.4f ( %.4f )    ※ カッコ内は全予測値を平均値で置き換えた場合\n' % (mse_pred, mse_mean, mae_pred, mae_mean))

# 売上高の予測の場合、これくらいになるはず
# 平均二乗誤差 : 0.0583 ( 0.0667 )  平均絶対誤差 : 0.1720 ( 0.1968 )

### 正解と予測の散布図を表示します。

In [0]:
sns.jointplot(y_test, y_pred, kind="reg").set_axis_labels('正解', '予測')

## LSTM

#### データを読み込みます。

In [0]:
df, y_column = read_data()

#### 3次元の入力の行列を作ります。 ( 会社数 x 4期 x 財務指標数 )

In [0]:
# 入力の列の名前
X_columns = [ x for x in df.columns if x != y_column ]

# 入力データ
dfX = df[X_columns]

# 出力データ
dfy = df[y_column]

# EDINETコード
edinet_codes = dfX.index.map(lambda x: x[0])

# 会社数
num_company = len(set(edinet_codes))

# 4期目までの値を使って、次期変化率を予測する。
years = 4

# Xの3次元行列  ( 会社数 x 4期 x 財務指標数 )
X_dt = np.zeros((num_company, years, len(X_columns) ))

# yの1次元配列
y_dt = np.zeros((num_company, ))

# 直前のEDINETコード
prev_code = None

# 現在の会社の開始行
start_row = 0

# 会社のインデックス
i_company = 0

# EDINETコードと次期変化率の値に対し
for row, (code, y) in enumerate(zip(edinet_codes, dfy)):
    if prev_code == code:
        # 会社が同じ場合

        if row - start_row == years - 1:
            # 現在の行が4期目の場合

            # 4期目までの値をXに入れる。
            X_dt[i_company, :, :] = dfX.values[start_row:start_row+years, :]

            # 4期目の次期変化率
            y_dt[i_company] = y

            # 会社のインデックス
            i_company += 1
    else:
        # 会社が変わった場合

        # 直前のEDINETコード
        prev_code = code

        # 現在の会社の開始行
        start_row = row

### データが入ってない部分を削除します。

In [0]:
X_dt = np.delete(X_dt, np.s_[i_company:], 0)
y_dt = np.delete(y_dt, np.s_[i_company:], 0)

### トレーニングデータとテストデータに分けます。

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_dt, y_dt, test_size=0.2, random_state=0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### LSTMのモデルを作ります。

In [0]:
unit = X_train.shape[2]
model = keras.Sequential([
    layers.LSTM(unit, dropout=0.5),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(unit, activation='selu'), # elu tanh sigmoid relu
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

### ログとEarlyStoppingのコールバックの設定をします。

In [0]:
log_dir = "logs/LSTM-" + datetime.datetime.now(pytz.timezone('Asia/Tokyo')).strftime("%H時%M分%S秒")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)

### 学習を開始します。

In [0]:
model.fit(x=X_train, y=y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[tensorboard_callback, early_stop])

### テストデータで予測します。

In [0]:
y_pred = model.predict(X_test)
y_pred = y_pred.reshape((y_pred.shape[0],))

#### 平均二乗誤差と平均絶対誤差を表示します。

In [0]:
# 平均二乗誤差を計算します。
mse_pred = mean_squared_error(y_test, y_pred)
mse_mean = mean_squared_error(y_test, [y_test.mean()] * len(y_test) )

# 平均絶対誤差を計算します。
mae_pred = mean_absolute_error(y_test, y_pred)
mae_mean = mean_absolute_error(y_test, [y_test.mean()] * len(y_test) )

print('\n平均二乗誤差 : %.4f ( %.4f )  平均絶対誤差 : %.4f ( %.4f )    ※ カッコ内は全予測値を平均値で置き換えた場合\n' % (mse_pred, mse_mean, mae_pred, mae_mean))

# 売上高の予測の場合、これくらいになるはず
# 平均二乗誤差 : 0.0435 ( 0.0542 )  平均絶対誤差 : 0.1481 ( 0.1713 )

### 正解と予測の散布図を表示します。

In [0]:
sns.jointplot(y_test, y_pred, kind="reg").set_axis_labels('正解', '予測')